In [1]:
import pandas as pd
import sqlite3
import math
import datetime 
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 44em; }</style>"))

In [2]:
conn = sqlite3.connect('database.db')

In [3]:
c = conn.cursor()

<img src="desain.png">

In [47]:
c.executescript('drop table if exists Transaksi')
c.executescript('drop table if exists Customer')
c.executescript('drop table if exists Product')
c.executescript('drop table if exists Category')
c.execute("CREATE TABLE IF NOT EXISTS 'Customer' ('id_customer' INT, 'cust_name' VARCHAR(255) NULL, 'city' VARCHAR(255) NULL, PRIMARY KEY ('id_customer'))")
c.execute("CREATE TABLE IF NOT EXISTS 'Category' ('id_category' INT, 'category' VARCHAR(255) NULL, PRIMARY KEY ('id_category'))")
c.execute("CREATE TABLE IF NOT EXISTS 'Product' ('id_product' INT, 'product_name' VARCHAR(255) NULL, 'id_category' INT NOT NULL, PRIMARY KEY ('id_product'), FOREIGN KEY ('id_category') REFERENCES 'Category' ('id_category'))")
c.execute("CREATE TABLE IF NOT EXISTS 'Transaksi' ('id_transaksi' INT, 'order_date' DATE NULL, 'ship_date' DATE NULL, 'quantity' VARCHAR(255) NULL, 'id_customer' INT NOT NULL, 'id_product' INT NOT NULL, PRIMARY KEY ('id_transaksi'), FOREIGN KEY ('id_customer') REFERENCES 'Customer' ('id_customer'), FOREIGN KEY ('id_product') REFERENCES 'Product' ('id_product'))")

In [48]:
df = pd.read_csv("Superstore (Tableau) - Orders.csv")
df

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country/Region,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2018-152156,11/8/2018,11/11/2018,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2018-152156,11/8/2018,11/11/2018,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2018-138688,6/12/2018,6/16/2018,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2017-108966,10/11/2017,10/18/2017,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2017-108966,10/11/2017,10/18/2017,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,9990,CA-2016-110422,1/21/2016,1/23/2016,Second Class,TB-21400,Tom Boeckenhauer,Consumer,United States,Miami,...,33180.0,South,FUR-FU-10001889,Furniture,Furnishings,Ultra Door Pull Handle,25.2480,3,0.20,4.1028
9990,9991,CA-2019-121258,2/26/2019,3/3/2019,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,92627.0,West,FUR-FU-10000747,Furniture,Furnishings,Tenex B1-RE Series Chair Mats for Low Pile Car...,91.9600,2,0.00,15.6332
9991,9992,CA-2019-121258,2/26/2019,3/3/2019,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,92627.0,West,TEC-PH-10003645,Technology,Phones,Aastra 57i VoIP phone,258.5760,2,0.20,19.3932
9992,9993,CA-2019-121258,2/26/2019,3/3/2019,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,92627.0,West,OFF-PA-10004041,Office Supplies,Paper,"It's Hot Message Books with Stickers, 2 3/4"" x 5""",29.6000,4,0.00,13.3200


In [49]:
Category = []
Product = []
Customer = []
Transaksi = []

for i,j in df[:100].iterrows():
    if j[-7] not in Category:
        Category.append(j[-7])
        
    ca = -1
    for index, value in enumerate(Category):
        if value == j[14]:
            ca = index+1
            break
            
            
    if [j[12],j[15], ca] not in Product:
        Product.append([j[12], j[15], ca])
        
    p = -1
    for index, value in enumerate(Product):
        if value[1] == j[15]:
            p = index+1
            break
            
    if [j[5],j[6], j[9]] not in Customer:
        Customer.append([j[5], j[6], j[9]])
        
    k = -1
    for index, value in enumerate(Customer):
        if value[0] == j[5]:
            k = index+1
            break
            
    if [j[0],j[2], j[3], j[17], k, p] not in Transaksi:
        Transaksi.append([j[5], j[6], j[9], k, p])
        
print(Category)
print(Product)
print(Customer)
print(Transaksi)

['Furniture', 'Office Supplies', 'Technology']
[['South', 'Bookcases', 1], ['South', 'Chairs', 1], ['West', 'Labels', 2], ['South', 'Tables', 1], ['South', 'Storage', 2], ['West', 'Furnishings', 1], ['West', 'Art', 2], ['West', 'Phones', 3], ['West', 'Binders', 2], ['West', 'Appliances', 2], ['West', 'Tables', 1], ['South', 'Paper', 2], ['Central', 'Appliances', 2], ['Central', 'Binders', 2], ['Central', 'Storage', 2], ['West', 'Storage', 2], ['Central', 'Art', 2], ['East', 'Chairs', 1], ['West', 'Accessories', 3], ['East', 'Bookcases', 1], ['East', 'Binders', 2], ['East', 'Furnishings', 1], ['East', 'Envelopes', 2], ['East', 'Art', 2], ['Central', 'Paper', 2], ['Central', 'Phones', 3], ['Central', 'Furnishings', 1], ['Central', 'Envelopes', 2], ['Central', 'Bookcases', 1], ['Central', 'Chairs', 1], ['Central', 'Accessories', 3], ['East', 'Accessories', 3], ['East', 'Phones', 3], ['Central', 'Labels', 2], ['East', 'Fasteners', 2], ['East', 'Storage', 2], ['East', 'Paper', 2], ['West', 

In [51]:
c.execute("DELETE FROM CATEGORY")
for index, i in enumerate(Category): 
    c.execute("INSERT INTO CATEGORY VALUES(" + str(index+1) + ", '" + str(i) + "')")
    
c.execute("DELETE FROM PRODUCT")
for index, i in enumerate(Product): 
    c.execute("INSERT INTO PRODUCT VALUES(" + str(index+1) + ", '" + str(i[0]) + "','" + str(i[1]).replace("'","") + "')")
    
c.execute("DELETE FROM CUSTOMER")
for index, i in enumerate(Customer): 
    c.execute("INSERT INTO CUSTOMER VALUES(" + str(index+1) + ", '" + str(i[0]) + "','" + str(i[1]).replace("'","") + "')")

c.execute("DELETE FROM TRANSAKSI")
for index, i in enumerate(Transaksi):
    id_customer = i[3]
    id_product = i[4]
    c.execute("INSERT INTO TRANSAKSI VALUES(" + str(index+1) + ", '" + str(i[0]) + "', '" + str(i[1]).replace("'","") + "', '" + str(i[2]) + "', " + str(id_customer) + ", " + str(id_product) + ")")


conn.commit()

In [53]:
c.execute('''SELECT * FROM CATEGORY''')

CATEGORY = []
for row in c.fetchall():
    CATEGORY.append(row)
print("========================CATEGORY================================")
print(CATEGORY)
print("============================================================")

c.execute('''SELECT * FROM PRODUCT''')

PRODUCT = []
for row in c.fetchall():
    PRODUCT.append(row)
print("========================PRODUCT================================")
print(PRODUCT)
print("============================================================")

c.execute('''SELECT * FROM CUSTOMER''')

CUSTOMER = []
for row in c.fetchall():
    CUSTOMER.append(row)
print("========================CUSTOMER================================")
print(CUSTOMER)
print("============================================================")

TRANSAKSI = []
c.execute('''SELECT * FROM TRANSAKSI''')
for row in c.fetchall():
    TRANSAKSI.append(row)
print("========================TRANSAKSI================================")
print(TRANSAKSI)
print("============================================================")

========================CATEGORY================================
[(1, 'Furniture'), (2, 'Office Supplies'), (3, 'Technology')]
========================PRODUCT================================
[(1, 'South', 'Bookcases'), (2, 'South', 'Chairs'), (3, 'West', 'Labels'), (4, 'South', 'Tables'), (5, 'South', 'Storage'), (6, 'West', 'Furnishings'), (7, 'West', 'Art'), (8, 'West', 'Phones'), (9, 'West', 'Binders'), (10, 'West', 'Appliances'), (11, 'West', 'Tables'), (12, 'South', 'Paper'), (13, 'Central', 'Appliances'), (14, 'Central', 'Binders'), (15, 'Central', 'Storage'), (16, 'West', 'Storage'), (17, 'Central', 'Art'), (18, 'East', 'Chairs'), (19, 'West', 'Accessories'), (20, 'East', 'Bookcases'), (21, 'East', 'Binders'), (22, 'East', 'Furnishings'), (23, 'East', 'Envelopes'), (24, 'East', 'Art'), (25, 'Central', 'Paper'), (26, 'Central', 'Phones'), (27, 'Central', 'Furnishings'), (28, 'Central', 'Envelopes'), (29, 'Central', 'Bookcases'), (30, 'Central', 'Chairs'), (31, 'Central', 'Accesso